# NHL Player Analysis

## Notebook Setup

To import the required python libraries and setting notebook display parameters.

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os.path as path
from sklearn import preprocessing
import warnings
import math
import glob
import os
warnings.simplefilter(action = 'ignore', category = FutureWarning)
warnings.simplefilter(action = 'ignore', category = UserWarning)

In [2]:
# Settings to allow all columns in dataframe to display
desired_width = 320
display_columns = 222
pd.set_option('display.width', desired_width)
np.set_printoptions(linewidth = desired_width)
pd.set_option('display.max_columns', display_columns)

## Loading Data

Loading NHL player data for multiple years into one master dataset.<br>
Source: https://www.moneypuck.com

In [3]:
def load_csv(folder_path):
    """Load multiple .csv files from a specific folder path"""
    
    # Return a list of paths matching a pathname pattern.
    all_files = glob.glob(folder_path + "/*.csv")

    # Initiate an empty list to store the data
    li = []

    # Loop to append each files data to the list
    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        li.append(df)

    # Join all the files into one DataFrame
    df = pd.concat(li, axis=0, ignore_index=True)

    df = df.sort_values(['name','season'], ascending=False)
    print('The shape of the data is:', df.shape)
    
    return df

In [4]:
# Local path to folder containing files for each year
path = '/Users/nathananderson/Documents/Data_Science/NHL_Player_Analysis/NHL_Player_Analysis/Datasets/nhl_player_data/'

df = load_csv(path)
df.head()

The shape of the data is: (47970, 154)


,playerId,season,name,team,position,situation,games_played,icetime,shifts,gameScore,onIce_xGoalsPercentage,offIce_xGoalsPercentage,onIce_corsiPercentage,offIce_corsiPercentage,onIce_fenwickPercentage,offIce_fenwickPercentage,iceTimeRank,I_F_xOnGoal,I_F_xGoals,I_F_xRebounds,I_F_xFreeze,I_F_xPlayStopped,I_F_xPlayContinuedInZone,I_F_xPlayContinuedOutsideZone,I_F_flurryAdjustedxGoals,I_F_scoreVenueAdjustedxGoals,I_F_flurryScoreVenueAdjustedxGoals,I_F_primaryAssists,I_F_secondaryAssists,I_F_shotsOnGoal,I_F_missedShots,I_F_blockedShotAttempts,I_F_shotAttempts,I_F_points,I_F_goals,I_F_rebounds,I_F_reboundGoals,I_F_freeze,I_F_playStopped,I_F_playContinuedInZone,I_F_playContinuedOutsideZone,I_F_savedShotsOnGoal,I_F_savedUnblockedShotAttempts,penalties,I_F_penalityMinutes,I_F_faceOffsWon,I_F_hits,I_F_takeaways,I_F_giveaways,I_F_lowDangerShots,I_F_mediumDangerShots,I_F_highDangerShots,I_F_lowDangerxGoals,I_F_mediumDangerxGoals,I_F_highDangerxGoals,I_F_lowDangerGoals,I_F_mediumDangerGoals,I_F_highDangerGoals,I_F_scoreAdjustedShotsAttempts,I_F_unblockedShotAttempts,I_F_scoreAdjustedUnblockedShotAttempts,I_F_dZoneGiveaways,I_F_xGoalsFromxReboundsOfShots,I_F_xGoalsFromActualReboundsOfShots,I_F_reboundxGoals,I_F_xGoals_with_earned_rebounds,I_F_xGoals_with_earned_rebounds_scoreAdjusted,I_F_xGoals_with_earned_rebounds_scoreFlurryAdjusted,I_F_shifts,I_F_oZoneShiftStarts,I_F_dZoneShiftStarts,I_F_neutralZoneShiftStarts,I_F_flyShiftStarts,I_F_oZoneShiftEnds,I_F_dZoneShiftEnds,I_F_neutralZoneShiftEnds,I_F_flyShiftEnds,faceoffsWon,faceoffsLost,timeOnBench,penalityMinutes,penalityMinutesDrawn,penaltiesDrawn,shotsBlockedByPlayer,OnIce_F_xOnGoal,OnIce_F_xGoals,OnIce_F_flurryAdjustedxGoals,OnIce_F_scoreVenueAdjustedxGoals,OnIce_F_flurryScoreVenueAdjustedxGoals,OnIce_F_shotsOnGoal,OnIce_F_missedShots,OnIce_F_blockedShotAttempts,OnIce_F_shotAttempts,OnIce_F_goals,OnIce_F_rebounds,OnIce_F_reboundGoals,OnIce_F_lowDangerShots,OnIce_F_mediumDangerShots,OnIce_F_highDangerShots,OnIce_F_lowDangerxGoals,OnIce_F_mediumDangerxGoals,OnIce_F_highDangerxGoals,OnIce_F_lowDangerGoals,OnIce_F_mediumDangerGoals,OnIce_F_highDangerGoals,OnIce_F_scoreAdjustedShotsAttempts,OnIce_F_unblockedShotAttempts,OnIce_F_scoreAdjustedUnblockedShotAttempts,OnIce_F_xGoalsFromxReboundsOfShots,OnIce_F_xGoalsFromActualReboundsOfShots,OnIce_F_reboundxGoals,OnIce_F_xGoals_with_earned_rebounds,OnIce_F_xGoals_with_earned_rebounds_scoreAdjusted,OnIce_F_xGoals_with_earned_rebounds_scoreFlurryAdjusted,OnIce_A_xOnGoal,OnIce_A_xGoals,OnIce_A_flurryAdjustedxGoals,OnIce_A_scoreVenueAdjustedxGoals,OnIce_A_flurryScoreVenueAdjustedxGoals,OnIce_A_shotsOnGoal,OnIce_A_missedShots,OnIce_A_blockedShotAttempts,OnIce_A_shotAttempts,OnIce_A_goals,OnIce_A_rebounds,OnIce_A_reboundGoals,OnIce_A_lowDangerShots,OnIce_A_mediumDangerShots,OnIce_A_highDangerShots,OnIce_A_lowDangerxGoals,OnIce_A_mediumDangerxGoals,OnIce_A_highDangerxGoals,OnIce_A_lowDangerGoals,OnIce_A_mediumDangerGoals,OnIce_A_highDangerGoals,OnIce_A_scoreAdjustedShotsAttempts,OnIce_A_unblockedShotAttempts,OnIce_A_scoreAdjustedUnblockedShotAttempts,OnIce_A_xGoalsFromxReboundsOfShots,OnIce_A_xGoalsFromActualReboundsOfShots,OnIce_A_reboundxGoals,OnIce_A_xGoals_with_earned_rebounds,OnIce_A_xGoals_with_earned_rebounds_scoreAdjusted,OnIce_A_xGoals_with_earned_rebounds_scoreFlurryAdjusted,OffIce_F_xGoals,OffIce_A_xGoals,OffIce_F_shotAttempts,OffIce_A_shotAttempts,xGoalsForAfterShifts,xGoalsAgainstAfterShifts,corsiForAfterShifts,corsiAgainstAfterShifts,fenwickForAfterShifts,fenwickAgainstAfterShifts
20255,8469820,2013,Zenon Konopka,BUF,C,other,59,307.0,9.0,-0.40,0.14,0.32,0.47,0.49,0.36,0.46,82.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,9.0,2.0,6.0,0.0,1.0,0.0,0.0,1.0,8.0,6.0,6.0,1790.0,0.0,0.0,0.0,0.0,2.99,0.15,0.13,0.15,0.13,1.0,3.0,3.0,7.0,0.0,1.0,0.0,4.0,0.0,0.0,0.15,0.00,0.00,0.

## Feature Engineering

To create and add new feature columns to the master DataFrame.

In [5]:
def new_features(df):
    """Add new feature columns to the DataFrame."""
    
    # New column for last season played
    df['Last_Season_Played'] = df.groupby(["playerId"])['season'].transform(np.max)
    
    df["% Games Played"] = df['games_played'] / 82
    df["Points per Game"] = df['I_F_points'] / df['games_played']
    df["Goals per Game"] = df['I_F_goals'] / df['games_played']
    df["Shots on Goal per Game"] = df['I_F_shotsOnGoal'] / df['games_played']
    #player_data["Primary Assists per Game"] = player_data['I_F_primaryAssists'] / player_data['games_played']
    #player_data["Secondary Assists per Game"] = player_data['I_F_secondaryAssists'] / player_data['games_played']
        
    return df

In [6]:
df = new_features(df)
df.head()

,playerId,season,name,team,position,situation,games_played,icetime,shifts,gameScore,onIce_xGoalsPercentage,offIce_xGoalsPercentage,onIce_corsiPercentage,offIce_corsiPercentage,onIce_fenwickPercentage,offIce_fenwickPercentage,iceTimeRank,I_F_xOnGoal,I_F_xGoals,I_F_xRebounds,I_F_xFreeze,I_F_xPlayStopped,I_F_xPlayContinuedInZone,I_F_xPlayContinuedOutsideZone,I_F_flurryAdjustedxGoals,I_F_scoreVenueAdjustedxGoals,I_F_flurryScoreVenueAdjustedxGoals,I_F_primaryAssists,I_F_secondaryAssists,I_F_shotsOnGoal,I_F_missedShots,I_F_blockedShotAttempts,I_F_shotAttempts,I_F_points,I_F_goals,I_F_rebounds,I_F_reboundGoals,I_F_freeze,I_F_playStopped,I_F_playContinuedInZone,I_F_playContinuedOutsideZone,I_F_savedShotsOnGoal,I_F_savedUnblockedShotAttempts,penalties,I_F_penalityMinutes,I_F_faceOffsWon,I_F_hits,I_F_takeaways,I_F_giveaways,I_F_lowDangerShots,I_F_mediumDangerShots,I_F_highDangerShots,I_F_lowDangerxGoals,I_F_mediumDangerxGoals,I_F_highDangerxGoals,I_F_lowDangerGoals,I_F_mediumDangerGoals,I_F_highDangerGoals,I_F_scoreAdjustedShotsAttempts,I_F_unblockedShotAttempts,I_F_scoreAdjustedUnblockedShotAttempts,I_F_dZoneGiveaways,I_F_xGoalsFromxReboundsOfShots,I_F_xGoalsFromActualReboundsOfShots,I_F_reboundxGoals,I_F_xGoals_with_earned_rebounds,I_F_xGoals_with_earned_rebounds_scoreAdjusted,I_F_xGoals_with_earned_rebounds_scoreFlurryAdjusted,I_F_shifts,I_F_oZoneShiftStarts,I_F_dZoneShiftStarts,I_F_neutralZoneShiftStarts,I_F_flyShiftStarts,I_F_oZoneShiftEnds,I_F_dZoneShiftEnds,I_F_neutralZoneShiftEnds,I_F_flyShiftEnds,faceoffsWon,faceoffsLost,timeOnBench,penalityMinutes,penalityMinutesDrawn,penaltiesDrawn,shotsBlockedByPlayer,OnIce_F_xOnGoal,OnIce_F_xGoals,OnIce_F_flurryAdjustedxGoals,OnIce_F_scoreVenueAdjustedxGoals,OnIce_F_flurryScoreVenueAdjustedxGoals,OnIce_F_shotsOnGoal,OnIce_F_missedShots,OnIce_F_blockedShotAttempts,OnIce_F_shotAttempts,OnIce_F_goals,OnIce_F_rebounds,OnIce_F_reboundGoals,OnIce_F_lowDangerShots,OnIce_F_mediumDangerShots,OnIce_F_highDangerShots,OnIce_F_lowDangerxGoals,OnIce_F_mediumDangerxGoals,OnIce_F_highDangerxGoals,OnIce_F_lowDangerGoals,OnIce_F_mediumDangerGoals,OnIce_F_highDangerGoals,OnIce_F_scoreAdjustedShotsAttempts,OnIce_F_unblockedShotAttempts,OnIce_F_scoreAdjustedUnblockedShotAttempts,OnIce_F_xGoalsFromxReboundsOfShots,OnIce_F_xGoalsFromActualReboundsOfShots,OnIce_F_reboundxGoals,OnIce_F_xGoals_with_earned_rebounds,OnIce_F_xGoals_with_earned_rebounds_scoreAdjusted,OnIce_F_xGoals_with_earned_rebounds_scoreFlurryAdjusted,OnIce_A_xOnGoal,OnIce_A_xGoals,OnIce_A_flurryAdjustedxGoals,OnIce_A_scoreVenueAdjustedxGoals,OnIce_A_flurryScoreVenueAdjustedxGoals,OnIce_A_shotsOnGoal,OnIce_A_missedShots,OnIce_A_blockedShotAttempts,OnIce_A_shotAttempts,OnIce_A_goals,OnIce_A_rebounds,OnIce_A_reboundGoals,OnIce_A_lowDangerShots,OnIce_A_mediumDangerShots,OnIce_A_highDangerShots,OnIce_A_lowDangerxGoals,OnIce_A_mediumDangerxGoals,OnIce_A_highDangerxGoals,OnIce_A_lowDangerGoals,OnIce_A_mediumDangerGoals,OnIce_A_highDangerGoals,OnIce_A_scoreAdjustedShotsAttempts,OnIce_A_unblockedShotAttempts,OnIce_A_scoreAdjustedUnblockedShotAttempts,OnIce_A_xGoalsFromxReboundsOfShots,OnIce_A_xGoalsFromActualReboundsOfShots,OnIce_A_reboundxGoals,OnIce_A_xGoals_with_earned_rebounds,OnIce_A_xGoals_with_earned_rebounds_scoreAdjusted,OnIce_A_xGoals_with_earned_rebounds_scoreFlurryAdjusted,OffIce_F_xGoals,OffIce_A_xGoals,OffIce_F_shotAttempts,OffIce_A_shotAttempts,xGoalsForAfterShifts,xGoalsAgainstAfterShifts,corsiForAfterShifts,corsiAgainstAfterShifts,fenwickForAfterShifts,fenwickAgainstAfterShifts,Last_Season_Played,% Games Played,Points per Game,Goals per Game,Shots on Goal per Game
20255,8469820,2013,Zenon Konopka,BUF,C,other,59,307.0,9.0,-0.40,0.14,0.32,0.47,0.49,0.36,0.46,82.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,9.0,2.0,6.0,0.0,1.0,0.0,0.0,1.0,8.0,6.0,6.0,1790.0,0.0,0.0,0.

## Slicing DataFrame

To slice the DataFrame based on selected criteria.

In [7]:
def slice_df(df):
    """Slice the DataFrame based on specific criteria."""

    df = df[df['situation'] == 'all']
    #df = df[df['I_F_points'] >= 10]
    df = df[df['Last_Season_Played'] ==  df['season'].max()]

    return df

In [8]:
df = slice_df(df)
df.head()

,playerId,season,name,team,position,situation,games_played,icetime,shifts,gameScore,onIce_xGoalsPercentage,offIce_xGoalsPercentage,onIce_corsiPercentage,offIce_corsiPercentage,onIce_fenwickPercentage,offIce_fenwickPercentage,iceTimeRank,I_F_xOnGoal,I_F_xGoals,I_F_xRebounds,I_F_xFreeze,I_F_xPlayStopped,I_F_xPlayContinuedInZone,I_F_xPlayContinuedOutsideZone,I_F_flurryAdjustedxGoals,I_F_scoreVenueAdjustedxGoals,I_F_flurryScoreVenueAdjustedxGoals,I_F_primaryAssists,I_F_secondaryAssists,I_F_shotsOnGoal,I_F_missedShots,I_F_blockedShotAttempts,I_F_shotAttempts,I_F_points,I_F_goals,I_F_rebounds,I_F_reboundGoals,I_F_freeze,I_F_playStopped,I_F_playContinuedInZone,I_F_playContinuedOutsideZone,I_F_savedShotsOnGoal,I_F_savedUnblockedShotAttempts,penalties,I_F_penalityMinutes,I_F_faceOffsWon,I_F_hits,I_F_takeaways,I_F_giveaways,I_F_lowDangerShots,I_F_mediumDangerShots,I_F_highDangerShots,I_F_lowDangerxGoals,I_F_mediumDangerxGoals,I_F_highDangerxGoals,I_F_lowDangerGoals,I_F_mediumDangerGoals,I_F_highDangerGoals,I_F_scoreAdjustedShotsAttempts,I_F_unblockedShotAttempts,I_F_scoreAdjustedUnblockedShotAttempts,I_F_dZoneGiveaways,I_F_xGoalsFromxReboundsOfShots,I_F_xGoalsFromActualReboundsOfShots,I_F_reboundxGoals,I_F_xGoals_with_earned_rebounds,I_F_xGoals_with_earned_rebounds_scoreAdjusted,I_F_xGoals_with_earned_rebounds_scoreFlurryAdjusted,I_F_shifts,I_F_oZoneShiftStarts,I_F_dZoneShiftStarts,I_F_neutralZoneShiftStarts,I_F_flyShiftStarts,I_F_oZoneShiftEnds,I_F_dZoneShiftEnds,I_F_neutralZoneShiftEnds,I_F_flyShiftEnds,faceoffsWon,faceoffsLost,timeOnBench,penalityMinutes,penalityMinutesDrawn,penaltiesDrawn,shotsBlockedByPlayer,OnIce_F_xOnGoal,OnIce_F_xGoals,OnIce_F_flurryAdjustedxGoals,OnIce_F_scoreVenueAdjustedxGoals,OnIce_F_flurryScoreVenueAdjustedxGoals,OnIce_F_shotsOnGoal,OnIce_F_missedShots,OnIce_F_blockedShotAttempts,OnIce_F_shotAttempts,OnIce_F_goals,OnIce_F_rebounds,OnIce_F_reboundGoals,OnIce_F_lowDangerShots,OnIce_F_mediumDangerShots,OnIce_F_highDangerShots,OnIce_F_lowDangerxGoals,OnIce_F_mediumDangerxGoals,OnIce_F_highDangerxGoals,OnIce_F_lowDangerGoals,OnIce_F_mediumDangerGoals,OnIce_F_highDangerGoals,OnIce_F_scoreAdjustedShotsAttempts,OnIce_F_unblockedShotAttempts,OnIce_F_scoreAdjustedUnblockedShotAttempts,OnIce_F_xGoalsFromxReboundsOfShots,OnIce_F_xGoalsFromActualReboundsOfShots,OnIce_F_reboundxGoals,OnIce_F_xGoals_with_earned_rebounds,OnIce_F_xGoals_with_earned_rebounds_scoreAdjusted,OnIce_F_xGoals_with_earned_rebounds_scoreFlurryAdjusted,OnIce_A_xOnGoal,OnIce_A_xGoals,OnIce_A_flurryAdjustedxGoals,OnIce_A_scoreVenueAdjustedxGoals,OnIce_A_flurryScoreVenueAdjustedxGoals,OnIce_A_shotsOnGoal,OnIce_A_missedShots,OnIce_A_blockedShotAttempts,OnIce_A_shotAttempts,OnIce_A_goals,OnIce_A_rebounds,OnIce_A_reboundGoals,OnIce_A_lowDangerShots,OnIce_A_mediumDangerShots,OnIce_A_highDangerShots,OnIce_A_lowDangerxGoals,OnIce_A_mediumDangerxGoals,OnIce_A_highDangerxGoals,OnIce_A_lowDangerGoals,OnIce_A_mediumDangerGoals,OnIce_A_highDangerGoals,OnIce_A_scoreAdjustedShotsAttempts,OnIce_A_unblockedShotAttempts,OnIce_A_scoreAdjustedUnblockedShotAttempts,OnIce_A_xGoalsFromxReboundsOfShots,OnIce_A_xGoalsFromActualReboundsOfShots,OnIce_A_reboundxGoals,OnIce_A_xGoals_with_earned_rebounds,OnIce_A_xGoals_with_earned_rebounds_scoreAdjusted,OnIce_A_xGoals_with_earned_rebounds_scoreFlurryAdjusted,OffIce_F_xGoals,OffIce_A_xGoals,OffIce_F_shotAttempts,OffIce_A_shotAttempts,xGoalsForAfterShifts,xGoalsAgainstAfterShifts,corsiForAfterShifts,corsiAgainstAfterShifts,fenwickForAfterShifts,fenwickAgainstAfterShifts,Last_Season_Played,% Games Played,Points per Game,Goals per Game,Shots on Goal per Game
5121,8476878,2022,Zemgus Girgensons,BUF,L,all,2,1477.0,37.0,1.07,0.41,0.46,0.42,0.46,0.45,0.45,19.0,5.53,0.44,0.34,1.09,0.18,3.20,2.75,0.44,0.43,0.43,0.0,0.0,4.0,4.0,1.0,9.0,1.0,1.0,2.0,0.0,0.0,0.0,3.0,2.0,3.0,7.0,1.0,2.0,3.0,1.0,0.0,0.0,5.0,3.0,0.0,0.13,0.31,0.00,1.0,0.0,0.0,8.93,8.0,7.97,0.0,0.07,0.35,0.00,0.51,0.50,0.50,37.0,1.0,6.0,10.0,20.0,4.0,5.0,7.0,21.0,3.0,1.0,5723.0,2.0

## PlayerID DataFrame

To get a DataFrame of players names and IDs to be used as a lookup table.

In [9]:
def playerid_df(df):
    """DataFrame for player names and IDs."""
    
    # Player Name & Player ID DataFrame
    players = df[['name', 'playerId','Last_Season_Played']].copy()
    
    # Removing duplicates
    players = players.drop_duplicates(subset=['playerId'], keep='first')
    players = players.sort_values('playerId').reset_index(drop = True)
    
    return players

In [10]:
players = playerid_df(df)
players.head()

,name,playerId,Last_Season_Played
0,Ryan Suter,8470600,2022
1,Jeff Carter,8470604,2022
2,Zach Parise,8470610,2022
3,Brent Burns,8470613,2022
4,Corey Perry,8470621,2022


## Machine Learning DataFrame

Create a DataFrame that is formatted to be used with various ML algorithms.

In [11]:
# DataFrame for ML
ml_df = df[['playerId', 'position', 'icetime',
            '% Games Played', 'Points per Game', 'Shots on Goal per Game',
            'Goals per Game', 'Shots on Goal per Game'
          ]].copy()

ml_df.head()

,playerId,position,icetime,% Games Played,Points per Game,Shots on Goal per Game,Goals per Game,Shots on Goal per Game
5121,8476878,L,1477.0,0.024390,0.500000,2.000000,0.500000,2.000000
26111,8476878,L,49692.0,0.682927,0.321429,1.642857,0.178571,1.642857
21346,8476878,C,57145.0,0.841463,0.275362,1.217391,0.173913,1.217391
34081,8476878,C,58768.0,0.878049,0.250000,1.083333,0.069444,1.083333
191,8476878,C,58404.0,0.865854,0.211268,1.591549,0.098592,1.591549


In [12]:
# Instantiate MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# Normalizing specific column so the values are between 0 and 1
ml_df[['icetime']] = scaler.fit_transform(ml_df[['icetime']])
ml_df[['Shots on Goal per Game']] = scaler.fit_transform(ml_df[['Shots on Goal per Game']])
ml_df.head()

,playerId,position,icetime,% Games Played,Points per Game,Shots on Goal per Game,Goals per Game,Shots on Goal per Game
5121,8476878,L,0.009607,0.024390,0.500000,0.285714,0.500000,0.285714
26111,8476878,L,0.343095,0.682927,0.321429,0.234694,0.178571,0.234694
21346,8476878,C,0.394645,0.841463,0.275362,0.173913,0.173913,0.173913
34081,8476878,C,0.405871,0.878049,0.250000,0.154762,0.069444,0.154762
191,8476878,C,0.403353,0.865854,0.211268,0.227364,0.098592,0.227364


In [13]:
"""
# OneHotEncode categorical columns
# OneHotEncode 'position'
onehot_position = pd.get_dummies(ml_df['position'])
ml_df = ml_df.join(onehot_position)
ml_df = ml_df.drop('position', axis=1)

# Rename onehot_position column names
pos_cols = {'C': 'Position - C',
            'D': 'Position - D',
            'L': 'Position - L',
            'R': 'Position - R'}

ml_df.rename(columns=pos_cols, inplace=True)

ml_df.head()
"""

"\n# OneHotEncode categorical columns\n# OneHotEncode 'position'\nonehot_position = pd.get_dummies(ml_df['position'])\nml_df = ml_df.join(onehot_position)\nml_df = ml_df.drop('position', axis=1)\n\n# Rename onehot_position column names\npos_cols = {'C': 'Position - C',\n            'D': 'Position - D',\n            'L': 'Position - L',\n            'R': 'Position - R'}\n\nml_df.rename(columns=pos_cols, inplace=True)\n\nml_df.head()\n"

In [14]:
# Instantiate LabelEncoder
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

# Label Encode 'position'
ml_df[['position']] = le.fit_transform(ml_df[['position']])

In [15]:
"""
# OneHotEncode 'name'
onehot_name = pd.get_dummies(ml_df['name'])
ml_df = ml_df.join(onehot_name)
ml_df = ml_df.drop('name', axis=1)

ml_df.head()
"""

"\n# OneHotEncode 'name'\nonehot_name = pd.get_dummies(ml_df['name'])\nml_df = ml_df.join(onehot_name)\nml_df = ml_df.drop('name', axis=1)\n\nml_df.head()\n"

In [16]:
ml_df.shape

(4184, 8)

In [17]:
# Separating features the ML models
features = ml_df.drop(labels= "Points per Game", axis=1)
features.head()

,playerId,position,icetime,% Games Played,Shots on Goal per Game,Goals per Game,Shots on Goal per Game
5121,8476878,2,0.009607,0.024390,0.285714,0.500000,0.285714
26111,8476878,2,0.343095,0.682927,0.234694,0.178571,0.234694
21346,8476878,0,0.394645,0.841463,0.173913,0.173913,0.173913
34081,8476878,0,0.405871,0.878049,0.154762,0.069444,0.154762
191,8476878,0,0.403353,0.865854,0.227364,0.098592,0.227364


In [18]:
# Separating labels the ML models
labels = ml_df["Points per Game"]
labels.head()

5121     0.500000
26111    0.321429
21346    0.275362
34081    0.250000
191      0.211268
Name: Points per Game, dtype: float64

In [19]:
# Splitting the data into training and testing datasets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.3, random_state = 10)

## Machine Learning

Predicting points per game by player using multiple machine learning models.

### Linear Regression

In [23]:
# Train a LinearRegression model
from sklearn.metrics import log_loss
from sklearn.linear_model import LinearRegression

lm = LinearRegression()
#lm.fit(X_train.values,y_train)
lm.fit(X_train, y_train)
print("** Linear Regression model has been trained.")
    
# Get the train and test accuracy scores
print(f"** Training Score: {lm.score(X_train, y_train)}")
print(f"** Testing Score: {lm.score(X_test, y_test)}")

** Linear Regression model has been trained.
** Training Score: 0.6759299592433701
** Testing Score: 0.6550569096277677


### Neural Network

### Predictions